### Environment Setup
We well access external data from ApacheSpark + setup the environment for Apache Spark


This notebook is designed to run in a IBM Watson Studio default runtime (NOT the Watson Studio Apache Spark Runtime as the default runtime with 1 vCPU is free of charge). Therefore, we install Apache Spark in local mode for test purposes only. Please don't use it in production.

If running outside Watson Studio, this should work as well. In case you are running in an Apache Spark context outside Watson Studio, please remove the Apache Spark setup in the first notebook cells.

In [1]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown('# <span style="color:red">'+string+'</span>'))


if ('sc' in locals() or 'sc' in globals()):
    printmd('<<<<<!!!!! It seems that you are running in a IBM Watson Studio Apache Spark Notebook. Please run it in an IBM Watson Studio Default Runtime (without Apache Spark) !!!!!>>>>>')


In [2]:
!pip install pyspark==3.3.0

     |████████████████████████████████| 281.3 MB 85 kB/s s eta 0:00:01
     |████████████████████████████████| 199 kB 79.6 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764027 sha256=4648731160b753b682d3ef8c7d153c49713d5357346805f23d46f29f91ef8ed2
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/05/75/73/81f84d174299abca38dd6a06a5b98b08ae25fce50ab8986fa1
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.2
    Uninstalling py4j-0.10.9.2:
      Successfully uninstalled py4j-0.10.9.2


In [3]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')

In [4]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/07 05:47:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [10]:
#implement a function returning the number of rows in the dataframe
def count():
    #some reference: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame
    #some more help: https://www.w3schools.com/sql/sql_count_avg_sum.asp
    return spark.sql('select count(*) as cnt from washing').first().cnt

Now let's return an integer containing the number of fields (columns). The most easy way to get this is using the dataframe API. 
You might find the dataframe API documentation useful: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/dataframe.html

In [11]:
def getNumberOfFields():
    #some reference: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame
    return len(df.columns)

Finally, let's implement a function which returns a (python) list of string values of the field names in this data frame. 
Again, this is the link to the documentation: http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame

In [12]:
def getFieldNames():
    #some reference: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame
    return df.columns

Now it is time to grab a PARQUET file and create a dataframe out of it. Using SparkSQL we can handle it like a database. 

In [13]:
!wget https://github.com/IBM/coursera/blob/master/coursera_ds/washing.parquet?raw=true
!mv washing.parquet?raw=true washing.parquet

--2023-03-07 05:55:17--  https://github.com/IBM/coursera/blob/master/coursera_ds/washing.parquet?raw=true
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/claimed-framework/component-library/blob/master/coursera_ds/washing.parquet?raw=true [following]
--2023-03-07 05:55:17--  https://github.com/claimed-framework/component-library/blob/master/coursera_ds/washing.parquet?raw=true
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/claimed-framework/component-library/raw/master/coursera_ds/washing.parquet [following]
--2023-03-07 05:55:18--  https://github.com/claimed-framework/component-library/raw/master/coursera_ds/washing.parquet
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.github

In [14]:
df = spark.read.parquet('washing.parquet')
df.createOrReplaceTempView('washing')
df.show()

+--------------------+--------------------+-----+--------+----------+---------+--------+-----+-----------+-------------+-------+
|                 _id|                _rev|count|flowrate|fluidlevel|frequency|hardness|speed|temperature|           ts|voltage|
+--------------------+--------------------+-----+--------+----------+---------+--------+-----+-----------+-------------+-------+
|0d86485d0f88d1f9d...|1-57940679fb8a713...|    4|      11|acceptable|     null|      77| null|        100|1547808723923|   null|
|0d86485d0f88d1f9d...|1-15ff3a0b304d789...|    2|    null|      null|     null|    null| 1046|       null|1547808729917|   null|
|0d86485d0f88d1f9d...|1-97c2742b68c7b07...|    4|    null|      null|       71|    null| null|       null|1547808731918|    236|
|0d86485d0f88d1f9d...|1-eefb903dbe45746...|   19|      11|acceptable|     null|      75| null|         86|1547808738999|   null|
|0d86485d0f88d1f9d...|1-5f68b4c72813c25...|    7|    null|      null|       75|    null| null|   

The following cell can be used to test our count function

In [15]:
cnt = None
nof = None
fn = None

cnt = count()
print(cnt)

2058


The following cell can be used to test our getNumberOfFields function

In [16]:
nof = getNumberOfFields()
print(nof)

11


The following cell can be used to test our getFieldNames function

In [17]:
fn = getFieldNames()
print(fn)

['_id', '_rev', 'count', 'flowrate', 'fluidlevel', 'frequency', 'hardness', 'speed', 'temperature', 'ts', 'voltage']


Congratulations, we are done